<a href="https://colab.research.google.com/github/ChangFeng23/4604Project/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 4.9 MB/s 
     |████████████████████████████████| 164 kB 42.5 MB/s 
     |████████████████████████████████| 78 kB 4.7 MB/s 
     |████████████████████████████████| 111 kB 53.4 MB/s 
     |████████████████████████████████| 232 kB 45.4 MB/s 
     |████████████████████████████████| 181 kB 39.4 MB/s 
     |████████████████████████████████| 4.3 MB 29.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 133 kB 46.3 MB/s 
     |████████████████████████████████| 132 kB 62.3 MB/s 
     |████████████████████████████████| 793 kB 477 kB/s 
     |████████████████████████████████| 428 kB 44.3 MB/s 
     |████████████████████████████████| 381 kB 43.5 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=34f81b01a17d

In [2]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=e7b2078b243a664ff10c392ef4b8ffc86d2330eb713c961f50d2287be23e8fec
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
!pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%%writefile app.py
import streamlit as st
import plotly_express as px
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
#configuration
st.set_option('deprecation.showfileUploaderEncoding', False)

#title of app
st.title("Data Visualization App")

#Add a sidebar
st.sidebar.subheader("Visualization Settings")

#Setup file upload
uploaded_file = st.sidebar.file_uploader(label="Upload files",
                         type=['csv','xlsx'])

global df
if uploaded_file is not None:
    print(uploaded_file)
    print("hello")
    try:
      df = pd.read_csv(uploaded_file)
    except Exception as e:
      print(e)
      df = pd.read_excel(uploaded_file)




try:
  st.write(df)
  numeric_columns = list(df.select_dtypes(['float', 'int']).columns)
  string_columns = list(df.select_dtypes(['object']).columns)
  yes = list(df.select_dtypes(['object', 'float', 'int']).columns)
except Exception as e:
  print(e)
  st.write("please upload file to the application")
  
df1 = df.drop(columns=['MMSE', 'GDS-15'])
GDS = df[['GDS-15']]
MMSE =df[['MMSE']]


categorical = ['STATE',\
               'GENDER',\
               'Marital Status',\
               'Living',\
               'Smoking',\
               'Drinking Alcohol',\
               'Job Sector Previously',\
                #'GDS-15',\
                #'MMSE',\
               'Average Total Neighbourhood',\
               'Employment Status',\
               'MOSSF(Informational)',\
               'MOSSF (Tangible Support)',\
               'MOSSF (Affective Support)',\
               'MOSSF (Positive Social Interaction)',\
               'Neighbourhood - General Feel',\
               'Total SWLS',\
               'Total_EpQ(Data_Full)(Average)',\
               'WHODAS_baseline',\
               'sumLubben',\
               'ADL',\
               'Total Social Cohesion Scale ',\
               'Total_Loneliness '
               
               ]



df = pd.get_dummies(df[categorical])




st.title('GDS and MMSE Prediction')
st.write("""
# Explore different classifier
""")
dataset_name = st.sidebar.selectbox("Select Dataset", ("GDS-15", "MMSE"))
st.write(dataset_name)
classifier_name = st.sidebar.selectbox("Select Classifier", ("KNN", "Decision Tree", "Naive Bayes"))
split_size = st.sidebar.slider('Data split ratio (% for Training Set)', 10, 90, 80, 5)

def get_dataset(dataset_name):
    if dataset_name == "GDS-15":
        target = GDS
    elif dataset_name == "MMSE":
        target = MMSE
    
    x = df
    y = target
    return x, y
X,y = get_dataset(dataset_name)
st.write("Shape of dataset", X.shape)
st.write("Number of Classes", len(np.unique(y)))

def add_parameter_ui(clp_name):
    params = dict()
    if clp_name == "KNN":
        K = st.sidebar.slider("K", 1, 15)
        params["K"] = K
    elif clp_name == "Naive Bayes":
        max_depth = st.sidebar.slider("max_depth", 2, 15)
        #n_estimators = st.sidebar.slider("n_estimators", 1, 100)
        params['max_depth'] = max_depth
        #params['n_estimators'] = n_estimators
    elif clp_name == "Decision Tree":
        max_depth = st.sidebar.slider("max_depth", 2, 15)
        #n_estimators = st.sidebar.slider("n_estimators", 1, 100)
        params['max_depth'] = max_depth
        #params['n_estimators'] = n_estimators
    
    

    return params
params = add_parameter_ui(classifier_name)

def get_classifier(clf_name, params):
    if clf_name == "KNN":
        clf = KNeighborsClassifier(n_neighbors=params["K"])
    elif clf_name == "Naive Bayes":
         clf = GaussianNB()
    else:
        clf= DecisionTreeClassifier(max_depth=params["max_depth"], random_state=1234)
    return clf

clf = get_classifier(classifier_name, params)

#Classfication
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size=split_size, random_state=1234)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
st.write(f"classifier  = {classifier_name}")
st.write(f"accuracy ={acc}")
cm_dtc = confusion_matrix(y_test, y_pred)
st.write('Confusion matrix: ', cm_dtc)
# Plot
pca = PCA(2)
X_projected = pca.fit_transform(X)
x1 = X_projected[:, 0]
x2 = X_projected[:, 1]

fig = plt.figure()
plt.scatter(x1, x2, alpha=0.8, cmap="viridis")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar()

# plt.show from normal
st.pyplot(fig)











Writing app.py


In [5]:
#check the file is ready
!ls

app.py	sample_data


In [6]:
#start the app at port 80
#!nohub streamlit run app.py &
!streamlit run --server.port 80 app.py&>/dev/null&

In [7]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

<NgrokTunnel: "http://d212-35-237-151-238.ngrok.io" -> "http://localhost:80">

In [8]:
#shutting down app
!pgrep streamlit

248


In [9]:
!ps -eaf | grep streamlit

root         248       1  8 17:17 ?        00:00:03 /usr/bin/python3 /usr/local/bin/streamlit run --server.port 80 app.py
root         277      63  0 17:18 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root         279     277  0 17:18 ?        00:00:00 grep streamlit


In [10]:
ngrok.kill()